In [1]:
import os
from KeyFrameExtractorClass import KeyFrameExtractor
from Points2VecClass import Point2Vec
import cv2
from VideoFormaterClass import VideoFormater

In [2]:
def obtain_paths(root, extension):
    paths = []

    for filename in os.listdir(root):
        if filename.endswith(extension):
            paths.append(os.path.join(root, filename))
    return paths

In [3]:
base_path = os.getcwd().split("\\")
project_directory = ""
for part in base_path:
    if part != "Modules":
        project_directory += part
        project_directory += "\\"
root_training_vids = project_directory + "Resources\\Videos\\Training\\raw_videos"
root_validation_vids = project_directory + "Resources\\Videos\\Validation\\raw_videos"
root_test_vids = project_directory + "Resources\\Videos\\Test\\raw_videos"
root_translation_csv = project_directory + "Resources\\Translations"

In [5]:
#training_vids = obtain_paths(root_training_vids, '.mp4')
validation_vids = obtain_paths(root_validation_vids, '.mp4')
test_vids = obtain_paths(root_test_vids, '.mp4')
translation_csv = obtain_paths(root_translation_csv, '.csv')
print("validation_vids:" + str(len(validation_vids)))
print("test_vids:" + str(len(test_vids)))
print("translation_csv:")
for index, file in enumerate(translation_csv):
    print(f"{index}: {file.split('\\')[-1].split('.')[0].split('_')[-1]}")

validation_vids:132
test_vids:184
translation_csv:
0: test
1: train
2: val


In [6]:
def make_df_video(video_paths, csv_path, type):
    kfe = KeyFrameExtractor()
    videoFormater = VideoFormater()
    normalizer = Point2Vec(4)
    translationDf = videoFormater.csvToTranslationDf(csv_path)
    os.mkdir(project_directory + f"Resources\\Datasets\\{type}")
    for index, path in enumerate(video_paths):
        print("Processing video: ", index)
        video = cv2.VideoCapture(path)
        file_name = path.split(os.path.sep)[-1].split(".mp4")[0]
        translation_filter = translationDf["VIDEO_NAME"] == file_name
        translation = translationDf[translation_filter]["SENTENCE"].iloc[0]
        video_points = kfe.extractKeyFrames(return_frame=False, draw=False, video=video)
        landmarks = normalizer.land2vec(video_points)
        cant_keyframes = len(video_points)
        dict = {"points": landmarks, "translation": translation, "id": index, "len_keyframes": cant_keyframes}
        dataFrame = videoFormater.formatVideo(dict)
        videoFormater.concatAndExportVideos(dataFrame, project_directory + f"Resources\\Datasets\\{type}\\{file_name}.csv")

In [7]:
def processVideos(all_paths, all_csv_paths, folder_names):
    print(all_paths, all_csv_paths, folder_names)
    for video_paths, csv_path, folder_name in zip(all_paths, all_csv_paths, folder_names):
        if len(video_paths) == 0:
            continue
        make_df_video(video_paths, csv_path, folder_name)

In [8]:
processVideos([validation_vids, test_vids], [translation_csv[2], translation_csv[0]], ["Validation", "Test"])

[['d:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\-d5dN54tH2E-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\-f1_kdl050s-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\00dWJ4YRRSI-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\0oGfy530AuI-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\0pKzG0RRUz4-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\0pKzG0RRUz4-2-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\0zvsqf23tmw-1-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\0zvsqf23tmw-2-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\1-xK5UtDSmE-2-rgb_front.mp4', 'd:\\SignAI-ML\\AI-Module\\Resources\\Videos\\Validation\\raw_videos\\11JT4jRNI-o-2-rgb_front.mp4'

d:\SignAI-ML\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Threshold: $0.053253727513767696 
 Hands size: $0.007827910495145751
Threshold: $0.0304933677587563 
 Hands size: $0.00016583801194498292
Threshold: $0.03039477551022543 
 Hands size: $0.00013269770676980016
Threshold: $0.030231490392101962 
 Hands size: $7.781190785117228e-05
Threshold: $0.030377428981524485 
 Hands size: $0.00012686693372301067
